In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
df_location = pd.read_excel('../../python/df_location.xlsx')
df_distance = pd.read_excel('../../python/df_distance_km.xlsx')
df_vehicle = pd.read_excel('../../python/df_vehicle.xlsx')
df_orders = pd.read_excel('../../python/df_orders.xlsx')

In [3]:
clients_names = df_location['cliente'].to_list()
clients_coords = df_location[['Latitud','Longitud']].to_numpy()

clients = {name: coord for name, coord in zip(clients_names, clients_coords)}

In [4]:
distances_matrix = df_distance.to_numpy()

In [5]:
def calculate_route_distance(route, distance_matrix, cost_per_unit):
    distance = 0
    for i in range(len(route) - 1):
        dist = distance_matrix[route[i]][route[i + 1]]
        if dist == 0:
            return float('inf')
        distance += dist * cost_per_unit
    return distance

In [6]:
def is_valid_route(route, demandas, capacity):
    total_demand = sum(demandas[cliente] for cliente in route)
    return total_demand <= capacity

In [7]:
def generate_valid_initial_routes(distance_matrix, num_vehiculos, almacen, demandas, capacidades):
    clientes = [cliente for cliente in range(len(distance_matrix)) if cliente != almacen]
    valid_routes_found = False
    while not valid_routes_found:
        random.shuffle(clientes)
        routes = [[almacen] for _ in range(num_vehiculos)]
        for cliente in clientes:
            for route in routes:
                if sum(demandas[r] for r in route) + demandas[cliente] <= capacidades[routes.index(route)]:
                    route.append(cliente)
                    break
        for route in routes:
            route.append(almacen)
        if all(calculate_route_distance(route, distance_matrix, 1) != float('inf') for route in routes):
            valid_routes_found = True
    return routes


In [8]:
def calculate_total_distance(routes, distance_matrix, costes):
    total_distance = 0
    for route, cost in zip(routes, costes):
        total_distance += calculate_route_distance(route, distance_matrix, cost)
    return total_distance

In [9]:
def generate_vecinos_multiple(routes, almacen, demandas, capacidades):
    neighbors = []
    for i in range(len(routes)):
        for j in range(len(routes)):
            if i != j:
                for k in range(1, len(routes[i]) - 1):
                    for l in range(1, len(routes[j]) - 1):
                        neighbor = [route[:] for route in routes]
                        neighbor[i][k], neighbor[j][l] = neighbor[j][l], neighbor[i][k]
                        if is_valid_route(neighbor[i], demandas, capacidades[i]) and is_valid_route(neighbor[j], demandas, capacidades[j]):
                            neighbors.append(neighbor)
    return neighbors

In [10]:
def tabu_search_multiple_tsp(distance_matrix, initial_routes, max_iterations, tabu_size, almacen, costes, demandas, capacidades):
    current_routes = initial_routes
    best_routes = current_routes
    best_distance = calculate_total_distance(current_routes, distance_matrix, costes)
    tabu_list = []

    for iteration in range(max_iterations):
        neighbors = generate_vecinos_multiple(current_routes, almacen, demandas, capacidades)
        evaluated_neighbors = [(neighbor, calculate_total_distance(neighbor, distance_matrix, costes)) for neighbor in neighbors if neighbor not in tabu_list]

        if not evaluated_neighbors:
            print("No se encontraron rutas válidas en esta iteración.")
            break

        best_neighbor = min(evaluated_neighbors, key=lambda x: x[1])
        current_routes = best_neighbor[0]

        tabu_list.append(current_routes)
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)

        if best_neighbor[1] < best_distance:
            best_routes = current_routes
            best_distance = best_neighbor[1]

    return best_routes, best_distance


In [11]:
def convert_routes_to_city_names(routes, city_names):
    return [[city_names[city] for city in route] for route in routes]

In [ ]:
almacen = len(clients_names) - 1
num_vehicles = 5
costes = df_vehicle['costo_km'].to_list()
capacidades = df_vehicle['capacidad_kg'].to_list()

demandas_dict = dict(zip(df_orders["cliente"], df_orders["order_demand"]))
demandas = [demandas_dict.get(client, 0) for client in clients_names]

initial_routes = generate_valid_initial_routes(distances_matrix, num_vehicles, almacen, demandas, capacidades)
print(initial_routes)
max_iterations = 500
tabu_size = 5

# # Ejecutar Tabú Search para múltiples viajeros
best_routes, best_distance = tabu_search_multiple_tsp(distances_matrix, initial_routes, max_iterations, tabu_size, almacen, costes, demandas, capacidades)

# # Convertir rutas a nombres de ciudades
best_routes_named = convert_routes_to_city_names(best_routes, clients_names)

print(f"\nMejores rutas encontradas: {best_routes_named}, Costo total: {best_distance}")
